In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Medium_accounts.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/LI-Small_Trans.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Large_accounts.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Large_Trans.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Medium_Trans.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/LI-Medium_accounts.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Small_Patterns.txt
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Medium_Patterns.txt
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Small_accounts.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/LI-Medium_Trans.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Large_Patterns.txt
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/LI-Medium_Patterns.txt
/kaggle/in

In [2]:
# !pip install dgl

In [3]:
!pip install --no-cache-dir dgl==1.1.2

In [4]:
import dgl
import torch
from collections import defaultdict
import pandas as pd

In [5]:
df_account = pd.read_csv("/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Small_accounts.csv")
df_account.shape

(518581, 5)

In [6]:
df_trans = pd.read_csv("/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Small_Trans.csv")
df_trans.shape

(5078345, 11)

In [7]:
df_joined = pd.merge(
    df_trans,
    df_account,
    how='left',
    left_on='Account',
    right_on='Account Number'
)
df_joined.drop(columns=['Account Number'], inplace=True)

In [8]:
df_joined.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Bank Name,Bank ID,Entity ID,Entity Name
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0,National Bank of Laramie,10,800D232D0,Partnership #1
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,Sappo Cooperative Bank,3208,8008EEA70,Partnership #2
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0,National Bank of Fort Wayne,3209,800FBB3A0,Partnership #3
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0,National Bank of the East,12,800C0EF20,Sole Proprietorship #1
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0,National Bank of Laramie,10,800C3EC10,Partnership #4


In [9]:
from_bank_counts_df = df_joined['From Bank'].value_counts().reset_index()
from_bank_counts_df.columns = ['From Bank', 'Jumlah']
print(from_bank_counts_df)

       From Bank  Jumlah
0             70  449859
1             10   81629
2             12   79754
3              1   62211
4             15   52511
...          ...     ...
30465     352684       1
30466     352696       1
30467     352626       1
30468     352764       1
30469     352766       1

[30470 rows x 2 columns]


In [10]:
df_joined = df_joined.drop(columns=['Bank Name', 'Receiving Currency', 'Payment Currency'])
top_5_banks = [70, 10, 12, 1, 15]

df_70 = df_joined[df_joined['From Bank'] == 70]
df_10 = df_joined[df_joined['From Bank'] == 10]
df_12 = df_joined[df_joined['From Bank'] == 12]
df_1 = df_joined[df_joined['From Bank'] == 1]
df_15 = df_joined[df_joined['From Bank'] == 15]


In [11]:
df_15.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Amount Paid', 'Payment Format', 'Is Laundering',
       'Bank ID', 'Entity ID', 'Entity Name'],
      dtype='object')

In [12]:
# df_70.to_csv('/kaggle/working/df_Oasis_Thrift.csv', index=True)
# df_10.to_csv('/kaggle/working/df_Laramie.csv', index=True)
# df_12.to_csv('/kaggle/working/df_East.csv', index=True)
# df_1.to_csv('/kaggle/working/df_Arbor_Savings.csv', index=True)
# df_15.to_csv('/kaggle/working/df_Japan.csv', index=True)

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import defaultdict
import pandas as pd
import torch
import dgl
import numpy as np


def preprocess_bank_graph(df_bank):
    edge_dict = defaultdict(list)

    # Konversi Timestamp ke datetime
    df_bank = df_bank.copy()
    df_bank['Timestamp'] = pd.to_datetime(df_bank['Timestamp'])
    df_bank['Timestamp_Seconds'] = (df_bank['Timestamp'] - df_bank['Timestamp'].min()).dt.total_seconds()

    account_ids = pd.Index(df_bank['Account'].tolist() + df_bank['Account.1'].tolist()).unique()
    account_id_map = {k: i for i, k in enumerate(account_ids)}
    bank_id_map = {k: i for i, k in enumerate(pd.Index(df_bank['From Bank'].tolist()).unique())}
    entity_id_map = {k: i for i, k in enumerate(pd.Index(df_bank['Entity ID'].tolist()).unique())}

    le_payment = LabelEncoder()
    df_bank['Payment Format Enc'] = le_payment.fit_transform(df_bank['Payment Format'])

    transfer_edge_feat = {
        'amount': [],
        'timestamp': [],
        'payment_format': [],
        'label': []
    }

    for _, row in df_bank.iterrows():
        src = account_id_map[row['Account']]
        dst = account_id_map[row['Account.1']]
        edge_dict[('account', 'transfers', 'account')].append((src, dst))
        transfer_edge_feat['amount'].append(row['Amount Paid'])
        transfer_edge_feat['timestamp'].append(row['Timestamp_Seconds'])
        transfer_edge_feat['payment_format'].append(row['Payment Format Enc'])
        transfer_edge_feat['label'].append(row['Is Laundering'])

        edge_dict[('account', 'belongs_to', 'bank')].append((src, bank_id_map[row['From Bank']]))
        edge_dict[('account', 'represents', 'entity')].append((src, entity_id_map[row['Entity ID']]))

    graph_data = {}
    for etype, edges in edge_dict.items():
        srcs, dsts = zip(*edges)
        graph_data[etype] = (torch.tensor(srcs), torch.tensor(dsts))

    g = dgl.heterograph(graph_data)

    g.edges['transfers'].data['amount'] = torch.tensor(transfer_edge_feat['amount'], dtype=torch.float32)
    g.edges['transfers'].data['timestamp'] = torch.tensor(transfer_edge_feat['timestamp'], dtype=torch.float32)
    g.edges['transfers'].data['payment_format'] = torch.tensor(transfer_edge_feat['payment_format'], dtype=torch.int64)
    g.edges['transfers'].data['label'] = torch.tensor(transfer_edge_feat['label'], dtype=torch.float32)

    # Buat label node
    account_labels = np.zeros(g.num_nodes('account'), dtype=np.int64)
    for (src, dst), label in zip(edge_dict[('account', 'transfers', 'account')], transfer_edge_feat['label']):
        if label == 1:
            account_labels[src] = 1
            account_labels[dst] = 1

    g.nodes['account'].data['y'] = torch.tensor(account_labels, dtype=torch.long)

    # Train/val/test split
    node_ids = np.arange(g.num_nodes('account'))
    labels = account_labels

    train_ids, test_ids = train_test_split(node_ids, test_size=0.2, random_state=42, stratify=labels)
    train_ids, val_ids = train_test_split(train_ids, test_size=0.1, random_state=42, stratify=labels[train_ids])

    # Init mask
    train_mask = np.zeros(g.num_nodes('account'), dtype=bool)
    val_mask = np.zeros(g.num_nodes('account'), dtype=bool)
    test_mask = np.zeros(g.num_nodes('account'), dtype=bool)

    train_mask[train_ids] = True
    val_mask[val_ids] = True
    test_mask[test_ids] = True

    g.nodes['account'].data['train_mask'] = torch.tensor(train_mask)
    g.nodes['account'].data['val_mask'] = torch.tensor(val_mask)
    g.nodes['account'].data['test_mask'] = torch.tensor(test_mask)

    return g


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
from dgl.nn import HeteroGraphConv
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import copy

In [ ]:
graph12 = preprocess_bank_graph(df_12)

In [ ]:
# # Menampilkan semua tipe node
# print(graph0.ntypes)

# # Menampilkan jumlah dan ID node untuk setiap tipe node
# for ntype in graph0.ntypes:
#     print(f"Tipe node: {ntype}")
#     print(f"Jumlah node: {graph0.num_nodes(ntype)}")
#     print(f"ID node: {list(range(graph0.num_nodes(ntype)))}")  # default ID adalah 0 sampai num_nodes-1


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
from dgl.nn import HeteroGraphConv
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from collections import defaultdict
import pandas as pd
import copy
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

class HGTLayer(nn.Module):
    def __init__(self, in_dim, out_dim, num_heads, ntypes, etypes, dropout=0.2):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.num_heads = num_heads
        self.dropout = dropout
        self.ntype2idx = {ntype: i for i, ntype in enumerate(ntypes)}
        self.etype2idx = {etype: i for i, etype in enumerate(etypes)}

        # Node type embeddings
        self.node_type_embed = nn.Parameter(torch.randn(len(ntypes), in_dim))
        
        # Edge type embeddings  
        self.edge_type_embed = nn.Parameter(torch.randn(len(etypes), in_dim))
        
        # Linear transformations for each node type
        self.linear_q = nn.ModuleDict()
        self.linear_k = nn.ModuleDict()
        self.linear_v = nn.ModuleDict()
        self.linear_out = nn.ModuleDict()
        
        for ntype in ntypes:
            self.linear_q[ntype] = nn.Linear(in_dim, out_dim * num_heads, bias=False)
            self.linear_k[ntype] = nn.Linear(in_dim, out_dim * num_heads, bias=False)
            self.linear_v[ntype] = nn.Linear(in_dim, out_dim * num_heads, bias=False)
            self.linear_out[ntype] = nn.Linear(out_dim * num_heads, out_dim)
        
        # Edge attention
        self.edge_attention = nn.ModuleDict()
        for etype in etypes:
            self.edge_attention[etype] = nn.Linear(in_dim, num_heads, bias=False)
        
        self.dropout_layer = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(out_dim)
        
    def forward(self, g, h_dict):
        with g.local_scope():
            # Add node type embeddings
            for ntype in h_dict:
                if h_dict[ntype].dim() == 1:
                    h_dict[ntype] = h_dict[ntype].unsqueeze(1)
                type_idx = self.ntype2idx[ntype]
                h_dict[ntype] = h_dict[ntype] + self.node_type_embed[type_idx]
            
            # Store original features for residual connection
            h_orig = {ntype: h.clone() for ntype, h in h_dict.items()}
            
            # Compute Q, K, V for each node type
            q_dict = {}
            k_dict = {}
            v_dict = {}
            
            for ntype in h_dict:
                q_dict[ntype] = self.linear_q[ntype](h_dict[ntype]).view(-1, self.num_heads, self.out_dim)
                k_dict[ntype] = self.linear_k[ntype](h_dict[ntype]).view(-1, self.num_heads, self.out_dim)
                v_dict[ntype] = self.linear_v[ntype](h_dict[ntype]).view(-1, self.num_heads, self.out_dim)
            
            # Message passing
            out_dict = {}
            for ntype in h_dict:
                out_dict[ntype] = []
                
                # Self-attention
                q = q_dict[ntype]  # [N, H, D]
                k = k_dict[ntype]  # [N, H, D]
                v = v_dict[ntype]  # [N, H, D]
                
                # Compute attention scores
                attn = torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(self.out_dim)
                attn = F.softmax(attn, dim=-1)
                attn = self.dropout_layer(attn)
                
                # Apply attention to values
                out = torch.matmul(attn, v)  # [N, H, D]
                out = out.flatten(1)  # [N, H*D]
                out = self.linear_out[ntype](out)
                
                out_dict[ntype] = out
            
            # Apply residual connection and normalization
            for ntype in out_dict:
                if out_dict[ntype].shape == h_orig[ntype].shape:
                    out_dict[ntype] = self.norm(out_dict[ntype] + h_orig[ntype])
                else:
                    out_dict[ntype] = self.norm(out_dict[ntype])
            
            return out_dict

class HGTModel(nn.Module):
    def __init__(self, node_types, edge_types, num_nodes_dict, hidden_dim=64, num_heads=4, num_layers=2, dropout=0.2):
        super().__init__()
        self.node_types = node_types
        self.edge_types = edge_types
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Initialize node features based on type
        self.node_projections = nn.ModuleDict()
        for ntype in node_types:
            if ntype == 'account':
                # Account nodes will use aggregated features
                self.node_projections[ntype] = nn.Linear(4, hidden_dim)  # 4 features: degree, avg_amount, total_amount, unique_payment_formats
            else:
                num_nodes = num_nodes_dict[ntype]
                self.node_projections[ntype] = nn.Embedding(num_nodes, hidden_dim)

        self.hgt_layers = nn.ModuleList()
        for i in range(num_layers):
            self.hgt_layers.append(
                HGTLayer(hidden_dim, hidden_dim, num_heads, node_types, edge_types, dropout)
            )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 2)
        )

    def _compute_account_features(self, g):
        """Compute meaningful features for account nodes"""
        num_accounts = g.num_nodes('account')
        features = torch.zeros(num_accounts, 4)
        
        # Get transfer edges
        src_nodes, dst_nodes = g.edges(etype='transfers')
        amounts = g.edges['transfers'].data['amount']
        payment_formats = g.edges['transfers'].data['payment_format']
        
        # Compute features for each account
        for acc_id in range(num_accounts):
            # Out-degree (how many transfers sent)
            out_mask = (src_nodes == acc_id)
            in_mask = (dst_nodes == acc_id)
            
            degree = out_mask.sum().item() + in_mask.sum().item()
            
            # Average amount
            relevant_amounts = torch.cat([amounts[out_mask], amounts[in_mask]])
            avg_amount = relevant_amounts.mean().item() if len(relevant_amounts) > 0 else 0.0
            
            # Total amount
            total_amount = relevant_amounts.sum().item() if len(relevant_amounts) > 0 else 0.0
            
            # Unique payment formats
            relevant_formats = torch.cat([payment_formats[out_mask], payment_formats[in_mask]])
            unique_formats = len(torch.unique(relevant_formats)) if len(relevant_formats) > 0 else 0
            
            features[acc_id] = torch.tensor([degree, avg_amount, total_amount, unique_formats])
        
        return features
        
    def forward(self, g):
        # Initialize node features
        h_dict = {}
        
        # Account nodes - use computed features
        account_features = self._compute_account_features(g)
        h_dict['account'] = self.node_projections['account'](account_features)
        
        # Other node types - use embeddings
        for ntype in self.node_types:
            if ntype != 'account':
                num_nodes = g.num_nodes(ntype)
                node_ids = torch.arange(num_nodes)  # Fixed: use torch.arange instead of g.nodes()
                h_dict[ntype] = self.node_projections[ntype](node_ids)
        
        # Apply HGT layers
        for layer in self.hgt_layers:
            h_dict = layer(g, h_dict)
        
        # Classification on account nodes
        logits = self.classifier(h_dict['account'])
        return logits

class FraudDetector:
    def __init__(self, model_config=None):
        self.model_config = model_config or {
            'hidden_dim': 64,
            'num_heads': 4,
            'num_layers': 2,
            'dropout': 0.2,
            'lr': 0.01,
            'weight_decay': 5e-4,
            'use_class_weight': True,
            'focal_loss_alpha': 0.25,
            'focal_loss_gamma': 2.0
        }
        self.model = None
        self.optimizer = None
        self.class_weights = None
        
    def compute_class_weights(self, labels):
        """Compute class weights for imbalanced data"""
        labels_np = labels.cpu().numpy()
        classes = np.unique(labels_np)
        class_weights = compute_class_weight('balanced', classes=classes, y=labels_np)
        return torch.tensor(class_weights, dtype=torch.float32)
        
    def focal_loss(self, logits, labels, alpha=0.25, gamma=2.0):
        """Focal loss for imbalanced classification"""
        ce_loss = F.cross_entropy(logits, labels, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = alpha * (1 - pt) ** gamma * ce_loss
        return focal_loss.mean()
        
    def initialize_model(self, graph):
        """Initialize model based on graph structure"""
        node_types = graph.ntypes
        edge_types = graph.etypes
        
        # Compute class weights for imbalanced data
        if self.model_config['use_class_weight']:
            labels = graph.nodes['account'].data['y']
            train_mask = graph.nodes['account'].data['train_mask']
            self.class_weights = self.compute_class_weights(labels[train_mask])
            print(f"Class weights: {self.class_weights}")
        
        num_nodes_dict = {ntype: graph.num_nodes(ntype) for ntype in graph.ntypes}
        self.model = HGTModel(
            node_types=graph.ntypes,
            edge_types=graph.etypes,
            num_nodes_dict=num_nodes_dict,
            hidden_dim=self.model_config['hidden_dim'],
            num_heads=self.model_config['num_heads'],
            num_layers=self.model_config['num_layers'],
            dropout=self.model_config['dropout']
        )
        
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=self.model_config['lr'],
            weight_decay=self.model_config['weight_decay']
        )
        
    def train_epoch(self, graph):
        """Train model for one epoch"""
        self.model.train()
        self.optimizer.zero_grad()
        
        logits = self.model(graph)
        labels = graph.nodes['account'].data['y']
        train_mask = graph.nodes['account'].data['train_mask']
        
        # Use focal loss for imbalanced data
        loss = self.focal_loss(
            logits[train_mask], 
            labels[train_mask],
            alpha=self.model_config['focal_loss_alpha'],
            gamma=self.model_config['focal_loss_gamma']
        )
        
        # Alternative: weighted cross entropy
        # if self.class_weights is not None:
        #     loss = F.cross_entropy(logits[train_mask], labels[train_mask], weight=self.class_weights)
        # else:
        #     loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def evaluate(self, graph, mask_key='val_mask'):
        """Evaluate model performance with detailed metrics"""
        self.model.eval()
        with torch.no_grad():
            logits = self.model(graph)
            labels = graph.nodes['account'].data['y']
            mask = graph.nodes['account'].data[mask_key]
            
            pred = logits[mask].argmax(dim=1)
            true = labels[mask]
            
            # Calculate confusion matrix
            cm = confusion_matrix(true.cpu(), pred.cpu())
            
            # Extract TN, FP, FN, TP
            if cm.shape == (2, 2):
                tn, fp, fn, tp = cm.ravel()
            else:
                # Handle case where only one class is present
                if len(np.unique(true.cpu())) == 1:
                    if np.unique(true.cpu())[0] == 0:
                        tn, fp, fn, tp = len(true), 0, 0, 0
                    else:
                        tn, fp, fn, tp = 0, 0, len(true), 0
                else:
                    tn, fp, fn, tp = 0, 0, 0, 0
            
            # Calculate metrics
            acc = accuracy_score(true.cpu(), pred.cpu())
            prec = precision_score(true.cpu(), pred.cpu(), average='weighted', zero_division=0)
            rec = recall_score(true.cpu(), pred.cpu(), average='weighted', zero_division=0)
            f1 = f1_score(true.cpu(), pred.cpu(), average='weighted', zero_division=0)
            
            # Calculate class-specific metrics
            if tp + fn > 0:
                recall_fraud = tp / (tp + fn)  # Sensitivity
            else:
                recall_fraud = 0.0
                
            if tp + fp > 0:
                precision_fraud = tp / (tp + fp)  # Positive Predictive Value
            else:
                precision_fraud = 0.0
                
            if tn + fp > 0:
                specificity = tn / (tn + fp)  # True Negative Rate
            else:
                specificity = 0.0
            
            # ROC AUC
            probs = F.softmax(logits[mask], dim=1)
            if len(torch.unique(true)) > 1:
                auc = roc_auc_score(true.cpu(), probs[:, 1].cpu())
            else:
                auc = 0.0
            
            return {
                'accuracy': acc,
                'precision': prec,
                'recall': rec,
                'f1': f1,
                'auc': auc,
                'tn': tn,
                'fp': fp,
                'fn': fn,
                'tp': tp,
                'recall_fraud': recall_fraud,
                'precision_fraud': precision_fraud,
                'specificity': specificity,
                'confusion_matrix': cm
            }
    
    def train(self, graph, epochs=100, early_stopping=10, verbose=True):
        """Train the model with progress bar"""
        if self.model is None:
            self.initialize_model(graph)
        
        # Debug: Print initial data distribution
        labels = graph.nodes['account'].data['y']
        train_mask = graph.nodes['account'].data['train_mask']
        val_mask = graph.nodes['account'].data['val_mask']
        test_mask = graph.nodes['account'].data['test_mask']
        
        print(f"Data distribution:")
        print(f"  Total accounts: {len(labels)}")
        print(f"  Overall class distribution: {torch.bincount(labels)}")
        print(f"  Train samples: {train_mask.sum().item()}, class dist: {torch.bincount(labels[train_mask])}")
        print(f"  Val samples: {val_mask.sum().item()}, class dist: {torch.bincount(labels[val_mask])}")
        print(f"  Test samples: {test_mask.sum().item()}, class dist: {torch.bincount(labels[test_mask])}")
        
        fraud_ratio = labels[train_mask].float().mean()
        print(f"  Fraud ratio in training: {fraud_ratio:.4f}")
        
        best_val_f1 = 0
        best_val_auc = 0
        patience = 0
        
        # Training loop with progress bar
        pbar = tqdm(range(epochs), desc="Training", ncols=100)
        
        for epoch in pbar:
            # Training
            train_loss = self.train_epoch(graph)
            
            # Validation
            val_metrics = self.evaluate(graph, 'val_mask')
            
            # Use AUC as primary metric for imbalanced data
            if val_metrics['auc'] > best_val_auc:
                best_val_auc = val_metrics['auc']
                best_val_f1 = val_metrics['f1']
                patience = 0
                # Save best model
                self.best_model_state = copy.deepcopy(self.model.state_dict())
            else:
                patience += 1
            
            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{train_loss:.4f}',
                'Val_F1': f'{val_metrics["f1"]:.4f}',
                'Val_AUC': f'{val_metrics["auc"]:.4f}',
                'TP': val_metrics['tp'],
                'FP': val_metrics['fp'],
                'FN': val_metrics['fn'],
                'TN': val_metrics['tn']
            })
            
            if verbose and epoch % 20 == 0:
                print(f"\nEpoch {epoch}:")
                print(f"  Loss: {train_loss:.4f}")
                print(f"  Val Metrics - F1: {val_metrics['f1']:.4f}, AUC: {val_metrics['auc']:.4f}")
                print(f"  Confusion Matrix - TN: {val_metrics['tn']}, FP: {val_metrics['fp']}, FN: {val_metrics['fn']}, TP: {val_metrics['tp']}")
                print(f"  Fraud Detection - Precision: {val_metrics['precision_fraud']:.4f}, Recall: {val_metrics['recall_fraud']:.4f}, Specificity: {val_metrics['specificity']:.4f}")
            
            if patience >= early_stopping:
                if verbose:
                    print(f"\nEarly stopping at epoch {epoch}")
                break
        
        pbar.close()
        
        # Load best model
        self.model.load_state_dict(self.best_model_state)
        
        # Final evaluation
        test_metrics = self.evaluate(graph, 'test_mask')
        if verbose:
            print(f"\n=== Final Test Results ===")
            print(f"Accuracy: {test_metrics['accuracy']:.4f}")
            print(f"F1 Score: {test_metrics['f1']:.4f}")
            print(f"AUC: {test_metrics['auc']:.4f}")
            print(f"Confusion Matrix:")
            print(f"  TN: {test_metrics['tn']}, FP: {test_metrics['fp']}")
            print(f"  FN: {test_metrics['fn']}, TP: {test_metrics['tp']}")
            print(f"Fraud Detection Metrics:")
            print(f"  Precision (PPV): {test_metrics['precision_fraud']:.4f}")
            print(f"  Recall (Sensitivity): {test_metrics['recall_fraud']:.4f}")
            print(f"  Specificity: {test_metrics['specificity']:.4f}")
        
        return test_metrics
    
    def get_model_parameters(self):
        """Get model parameters for federated learning"""
        return copy.deepcopy(self.model.state_dict())
    
    def set_model_parameters(self, parameters):
        """Set model parameters from federated learning"""
        self.model.load_state_dict(parameters)
    
    def predict(self, graph):
        """Make predictions on graph"""
        self.model.eval()
        with torch.no_grad():
            logits = self.model(graph)
            probabilities = F.softmax(logits, dim=1)
            predictions = logits.argmax(dim=1)
            
            return predictions, probabilities

def preprocess_bank_graph(df_bank):
    """Fixed preprocessing function"""
    edge_dict = defaultdict(list)
    
    # Convert Timestamp to datetime
    df_bank = df_bank.copy()
    df_bank['Timestamp'] = pd.to_datetime(df_bank['Timestamp'])
    df_bank['Timestamp_Seconds'] = (df_bank['Timestamp'] - df_bank['Timestamp'].min()).dt.total_seconds()
    
    # Create ID mappings
    account_ids = pd.Index(df_bank['Account'].tolist() + df_bank['Account.1'].tolist()).unique()
    account_id_map = {k: i for i, k in enumerate(account_ids)}
    
    bank_ids = pd.Index(df_bank['From Bank'].tolist()).unique()
    bank_id_map = {k: i for i, k in enumerate(bank_ids)}
    
    entity_ids = pd.Index(df_bank['Entity ID'].tolist()).unique()
    entity_id_map = {k: i for i, k in enumerate(entity_ids)}
    
    # Encode payment format
    le_payment = LabelEncoder()
    df_bank['Payment Format Enc'] = le_payment.fit_transform(df_bank['Payment Format'])
    
    # Debug: Print data statistics
    print(f"Preprocessing Debug:")
    print(f"  Total transactions: {len(df_bank)}")
    print(f"  Unique accounts: {len(account_ids)}")
    print(f"  Unique banks: {len(bank_ids)}")
    print(f"  Unique entities: {len(entity_ids)}")
    print(f"  Laundering transactions: {df_bank['Is Laundering'].sum()}")
    print(f"  Laundering ratio: {df_bank['Is Laundering'].mean():.4f}")
    
    transfer_edge_feat = {
        'amount': [],
        'timestamp': [],
        'payment_format': [],
        'label': []
    }
    
    # Build edges
    for _, row in df_bank.iterrows():
        src = account_id_map[row['Account']]
        dst = account_id_map[row['Account.1']]
        
        # Transfer edges
        edge_dict[('account', 'transfers', 'account')].append((src, dst))
        transfer_edge_feat['amount'].append(row['Amount Paid'])
        transfer_edge_feat['timestamp'].append(row['Timestamp_Seconds'])
        transfer_edge_feat['payment_format'].append(row['Payment Format Enc'])
        transfer_edge_feat['label'].append(row['Is Laundering'])
        
        # Bank and entity relationships
        edge_dict[('account', 'belongs_to', 'bank')].append((src, bank_id_map[row['From Bank']]))
        edge_dict[('account', 'represents', 'entity')].append((src, entity_id_map[row['Entity ID']]))
    
    # Create heterograph
    graph_data = {}
    for etype, edges in edge_dict.items():
        srcs, dsts = zip(*edges)
        graph_data[etype] = (torch.tensor(srcs), torch.tensor(dsts))
    
    g = dgl.heterograph(graph_data)
    
    # Add edge features
    g.edges['transfers'].data['amount'] = torch.tensor(transfer_edge_feat['amount'], dtype=torch.float32)
    g.edges['transfers'].data['timestamp'] = torch.tensor(transfer_edge_feat['timestamp'], dtype=torch.float32)
    g.edges['transfers'].data['payment_format'] = torch.tensor(transfer_edge_feat['payment_format'], dtype=torch.int64)
    g.edges['transfers'].data['label'] = torch.tensor(transfer_edge_feat['label'], dtype=torch.float32)
    
    # Create account node labels - Fixed logic
    account_labels = np.zeros(g.num_nodes('account'), dtype=np.int64)
    
    # Mark accounts involved in laundering transactions
    for (src, dst), label in zip(edge_dict[('account', 'transfers', 'account')], transfer_edge_feat['label']):
        if label == 1:  # If this is a laundering transaction
            account_labels[src] = 1  # Mark source account as fraudulent
            account_labels[dst] = 1  # Mark destination account as fraudulent
    
    g.nodes['account'].data['y'] = torch.tensor(account_labels, dtype=torch.long)
    
    # Debug: Print label distribution
    print(f"  Account labels - Class 0: {(account_labels == 0).sum()}, Class 1: {(account_labels == 1).sum()}")
    
    # Train/val/test split with stratification
    node_ids = np.arange(g.num_nodes('account'))
    labels = account_labels
    
    # Check if we have enough samples of each class
    unique_labels, counts = np.unique(labels, return_counts=True)
    print(f"  Label distribution: {dict(zip(unique_labels, counts))}")
    
    if len(unique_labels) > 1 and min(counts) >= 3:  # Need at least 3 samples for stratified split
        train_ids, test_ids = train_test_split(node_ids, test_size=0.2, random_state=42, stratify=labels)
        train_ids, val_ids = train_test_split(train_ids, test_size=0.1, random_state=42, stratify=labels[train_ids])
    else:
        # If not enough samples for stratification, use random split
        print("  Warning: Not enough samples for stratified split, using random split")
        train_ids, test_ids = train_test_split(node_ids, test_size=0.2, random_state=42)
        train_ids, val_ids = train_test_split(train_ids, test_size=0.1, random_state=42)
    
    # Create masks
    train_mask = np.zeros(g.num_nodes('account'), dtype=bool)
    val_mask = np.zeros(g.num_nodes('account'), dtype=bool)
    test_mask = np.zeros(g.num_nodes('account'), dtype=bool)
    
    train_mask[train_ids] = True
    val_mask[val_ids] = True
    test_mask[test_ids] = True
    
    g.nodes['account'].data['train_mask'] = torch.tensor(train_mask)
    g.nodes['account'].data['val_mask'] = torch.tensor(val_mask)
    g.nodes['account'].data['test_mask'] = torch.tensor(test_mask)
    
    return g

# Federated Learning utilities
class FederatedLearning:
    def __init__(self, num_clients):
        self.num_clients = num_clients
        self.global_model_params = None
        
    def federated_averaging(self, client_params_list, client_weights=None):
        """Perform federated averaging of model parameters"""
        if client_weights is None:
            client_weights = [1.0 / len(client_params_list)] * len(client_params_list)
        
        # Initialize global parameters
        global_params = {}
        
        # Average parameters
        for key in client_params_list[0].keys():
            global_params[key] = torch.zeros_like(client_params_list[0][key])
            
            for i, client_params in enumerate(client_params_list):
                global_params[key] += client_weights[i] * client_params[key]
        
        self.global_model_params = global_params
        return global_params
    
    def get_global_parameters(self):
        """Get global model parameters"""
        return self.global_model_params

# Example usage function
def train_single_bank(graph, bank_id, config=None):
    """Train model for a single bank"""
    print(f"\n=== Training model for Bank {bank_id} ===")
    
    # Enhanced config for imbalanced data
    if config is None:
        config = {
            'hidden_dim': 64,
            'num_heads': 4,
            'num_layers': 2,
            'dropout': 0.3,
            'lr': 0.001,  # Lower learning rate
            'weight_decay': 1e-4,
            'use_class_weight': True,
            'focal_loss_alpha': 0.25,
            'focal_loss_gamma': 2.0
        }
    
    detector = FraudDetector(config)
    test_metrics = detector.train(graph, epochs=200, early_stopping=15, verbose=True)
    
    return detector

def federated_training(graphs, config=None, rounds=10):
    """Simulate federated training across multiple banks"""
    print("\n=== Starting Federated Training ===")
    
    # Enhanced config for imbalanced data
    if config is None:
        config = {
            'hidden_dim': 64,
            'num_heads': 4,
            'num_layers': 2,
            'dropout': 0.3,
            'lr': 0.001,
            'weight_decay': 1e-4,
            'use_class_weight': True,
            'focal_loss_alpha': 0.25,
            'focal_loss_gamma': 2.0
        }
    
    # Initialize clients (banks)
    clients = []
    for i, graph in enumerate(graphs):
        detector = FraudDetector(config)
        detector.initialize_model(graph)
        clients.append(detector)
    
    # Initialize federated learning
    fl = FederatedLearning(len(clients))
    
    # Training rounds with progress bar
    round_pbar = tqdm(range(rounds), desc="Federated Rounds", ncols=100)
    
    for round_num in round_pbar:
        print(f"\n--- Federated Round {round_num + 1} ---")
        
        # Each client trains locally
        client_params = []
        round_metrics = []
        
        client_pbar = tqdm(enumerate(zip(clients, graphs)), 
                          total=len(clients), 
                          desc=f"Round {round_num + 1} - Client Training", 
                          ncols=100,
                          leave=False)
        
        for i, (client, graph) in client_pbar:
            client_pbar.set_description(f"Round {round_num + 1} - Training Bank {i+1}")
            
            # Train for a few epochs
            epoch_losses = []
            for epoch in range(5):
                loss = client.train_epoch(graph)
                epoch_losses.append(loss)
            
            # Get model parameters
            params = client.get_model_parameters()
            client_params.append(params)
            
            # Evaluate local model
            val_metrics = client.evaluate(graph, 'val_mask')
            round_metrics.append(val_metrics)
            
            # Update progress bar
            client_pbar.set_postfix({
                'Bank': i+1,
                'Loss': f'{np.mean(epoch_losses):.4f}',
                'Val_F1': f'{val_metrics["f1"]:.4f}',
                'Val_AUC': f'{val_metrics["auc"]:.4f}',
                'TP': val_metrics['tp'],
                'FP': val_metrics['fp']
            })
        
        client_pbar.close()
        
        # Print round summary
        print(f"\nRound {round_num + 1} Summary:")
        for i, metrics in enumerate(round_metrics):
            print(f"  Bank {i+1} - Val F1: {metrics['f1']:.4f}, AUC: {metrics['auc']:.4f}, "
                  f"TP: {metrics['tp']}, FP: {metrics['fp']}, FN: {metrics['fn']}, TN: {metrics['tn']}")
        
        # Calculate average metrics
        avg_f1 = np.mean([m['f1'] for m in round_metrics])
        avg_auc = np.mean([m['auc'] for m in round_metrics])
        avg_tp = np.mean([m['tp'] for m in round_metrics])
        avg_fp = np.mean([m['fp'] for m in round_metrics])
        
        # Update main progress bar
        round_pbar.set_postfix({
            'Avg_F1': f'{avg_f1:.4f}',
            'Avg_AUC': f'{avg_auc:.4f}',
            'Avg_TP': f'{avg_tp:.1f}',
            'Avg_FP': f'{avg_fp:.1f}'
        })
        
        # Federated averaging
        global_params = fl.federated_averaging(client_params)
        
        # Update all clients with global parameters
        for client in clients:
            client.set_model_parameters(global_params)
    
    round_pbar.close()
    
    # Final evaluation
    print("\n=== Final Evaluation ===")
    final_results = []
    
    eval_pbar = tqdm(enumerate(zip(clients, graphs)), 
                     total=len(clients), 
                     desc="Final Evaluation", 
                     ncols=100)
    
    for i, (client, graph) in eval_pbar:
        test_metrics = client.evaluate(graph, 'test_mask')
        final_results.append(test_metrics)
        
        eval_pbar.set_postfix({
            'Bank': i+1,
            'Test_F1': f'{test_metrics["f1"]:.4f}',
            'Test_AUC': f'{test_metrics["auc"]:.4f}',
            'TP': test_metrics['tp'],
            'FP': test_metrics['fp']
        })
        
        print(f"\nBank {i+1} Final Results:")
        print(f"  Test F1: {test_metrics['f1']:.4f}")
        print(f"  Test AUC: {test_metrics['auc']:.4f}")
        print(f"  Confusion Matrix - TN: {test_metrics['tn']}, FP: {test_metrics['fp']}, "
              f"FN: {test_metrics['fn']}, TP: {test_metrics['tp']}")
        print(f"  Fraud Detection - Precision: {test_metrics['precision_fraud']:.4f}, "
              f"Recall: {test_metrics['recall_fraud']:.4f}, Specificity: {test_metrics['specificity']:.4f}")
    
    eval_pbar.close()
    
    # Overall summary
    print(f"\n=== Overall Federated Learning Results ===")
    overall_f1 = np.mean([r['f1'] for r in final_results])
    overall_auc = np.mean([r['auc'] for r in final_results])
    overall_tp = np.sum([r['tp'] for r in final_results])
    overall_fp = np.sum([r['fp'] for r in final_results])
    overall_fn = np.sum([r['fn'] for r in final_results])
    overall_tn = np.sum([r['tn'] for r in final_results])
    
    print(f"Average F1 Score: {overall_f1:.4f}")
    print(f"Average AUC: {overall_auc:.4f}")
    print(f"Total Confusion Matrix:")
    print(f"  TN: {overall_tn}, FP: {overall_fp}")
    print(f"  FN: {overall_fn}, TP: {overall_tp}")
    
    if overall_tp + overall_fn > 0:
        overall_recall = overall_tp / (overall_tp + overall_fn)
        print(f"Overall Fraud Recall: {overall_recall:.4f}")
    
    if overall_tp + overall_fp > 0:
        overall_precision = overall_tp / (overall_tp + overall_fp)
        print(f"Overall Fraud Precision: {overall_precision:.4f}")
    
    if overall_tn + overall_fp > 0:
        overall_specificity = overall_tn / (overall_tn + overall_fp)
        print(f"Overall Specificity: {overall_specificity:.4f}")
    
    return clients, final_results

In [ ]:
detector = train_single_bank(graph0, bank_id=0)

In [ ]:
# bank_graphs = {
#     1: preprocess_bank_graph(df_1),
#     10: preprocess_bank_graph(df_10),
#     12: preprocess_bank_graph(df_12),
#     15: preprocess_bank_graph(df_15),
#     70: preprocess_bank_graph(df_70),
# }


In [ ]:
# for bank_id, graph in bank_graphs.items():
#     save_path = f"/kaggle/working/graph_bank_{bank_id}.bin"
#     dgl.save_graphs(save_path, [graph])
#     print(f"Saved graph for bank {bank_id} to {save_path}")

In [ ]:
# for bank_id, g in bank_graphs.items():
#     print(f"\n==== Bank {bank_id} ====")
#     for ntype in g.ntypes:
#         print(f"Node type: {ntype}")
#         for key in g.nodes[ntype].data.keys():
#             print(f"  - {key}: shape = {g.nodes[ntype].data[key].shape}")
